# Day 3 - Function calling with the Gemini API

## Setup

Start by installing and importing the Python SDK.

In [1]:
%pip install -q -U 'google-generativeai>=0.8.3'

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Create a local database

In [11]:
%load_ext sql
%sql sqlite:///sample.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2) NOT NULL
);

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL
);

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name VARCHAR(255) NOT NULL,
    staff_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
);

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
    ('Laptop', 799.99),
    ('Keyboard', 129.99),
    ('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
    ('Alice', 'Smith'),
    ('Bob', 'Johnson'),
    ('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
    ('David Lee', 1, 1),
    ('Emily Chen', 2, 2),
    ('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

## Define database functions
Start with a database connection that will be used across all of the functions.

In [48]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [49]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

In [50]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


print(describe_table("products"))
print(describe_table("staff"))
print(describe_table("orders"))

 - DB CALL: describe_table
[('product_id', 'INTEGER'), ('product_name', 'VARCHAR(255)'), ('price', 'DECIMAL(10, 2)')]
 - DB CALL: describe_table
[('staff_id', 'INTEGER'), ('first_name', 'VARCHAR(255)'), ('last_name', 'VARCHAR(255)')]
 - DB CALL: describe_table
[('order_id', 'INTEGER'), ('customer_name', 'VARCHAR(255)'), ('staff_id', 'INTEGER'), ('product_id', 'INTEGER')]


Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [51]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')
    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


print(f'All values in products: {execute_query("select * from products")}')
print(f'All values in staff: {execute_query("select * from staff")}')
print(f'All values in orders: {execute_query("select * from orders")}')

 - DB CALL: execute_query
All values in products: [(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]
 - DB CALL: execute_query
All values in staff: [(1, 'Alice', 'Smith'), (2, 'Bob', 'Johnson'), (3, 'Charlie', 'Williams')]
 - DB CALL: execute_query
All values in orders: [(1, 'David Lee', 1, 1), (2, 'Emily Chen', 2, 2), (3, 'Frank Brown', 1, 3)]


## Implement function calls

Now you can put it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a `function_call` instead. When this happens, the client must respond with a `function_response`, indicating the result of the call, and the conversation can continue on as normal.


In [57]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query. If the question is not related to the
information in the database, you can answer according to your knowledge from sources on the internet such as wikipedia or google.
"""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash-exp", tools=db_tools, system_instruction=instruction
)
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)

Now you can engage in a chat conversation where you can ask about the contents of the database.

In [60]:
response = chat.send_message('What is the cheapest product?', request_options=retry_policy)
print(response.text)

 - DB CALL: execute_query
The cheapest product is a Mouse, which costs 29.99.



In [61]:
resp = chat.send_message("and expensive ?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
The most expensive product is a Laptop, which costs 799.99.



## In a limited context within the database


In [62]:
resp = chat.send_message("Can you tell me why Laptop is so expensive ?", request_options=retry_policy)
print(resp.text)

The price of a laptop is influenced by a variety of factors, including the components used, the brand, and the features offered. Here's a breakdown of why laptops tend to be more expensive than other computer peripherals:

*   **Processors (CPU):** Laptops use powerful processors to handle demanding tasks. The more powerful the CPU, the more expensive it will be.
*   **Graphics Processing Unit (GPU):** For gaming or graphic-intensive tasks, a dedicated graphics card is essential, which adds to the cost.
*   **RAM (Memory):** The amount and speed of RAM affect how well a laptop can multitask. More RAM or faster RAM will cost more.
*   **Storage:** Solid-state drives (SSDs) are faster and more expensive than traditional hard drives (HDDs). The size of the storage also affects the cost.
*   **Screen:** The size, resolution, and panel technology (e.g., IPS, OLED) of the screen all contribute to the price.
*   **Build Quality:** Laptops made with premium materials like aluminum or carbon fi

In [63]:
response = chat.send_message('Which salesperson sold the cheapest product?', request_options=retry_policy)
print(response.text)

 - DB CALL: execute_query
The salesperson who sold the cheapest product was Alice Smith.



### Inspecting the conversation

To see the calls that the model makes, and what the client returns in response, you can inspect `chat.history`. This helper function will print out each turn along with the relevant fields passed or returned.

In [64]:
import textwrap

def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.history:
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(resp), "    "))

        print()


print_chat_turns(chat)

User:
  "Which salesperson sold the cheapest product?"

Model:
  Function call: list_tables()

User:
  Function response:
    name: "list_tables"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "products"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "staff"
            }
            values {
              string_value: "orders"
            }
          }
        }
      }
    }


Model:
  Function call: describe_table(table_name=products)
  "
"

User:
  Function response:
    name: "describe_table"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "product_id"
                }
                values {
                  string_value: "INTEGER"
            